In [1]:
# Retail Dataset: https://www.kaggle.com/datasets/sahilprajapati143/retail-analysis-large-dataset
# Holidays Dataset: https://www.kaggle.com/datasets/dhavalrupapara/world-countries-holidays-dataset-2023

In [2]:
import pandas as pd
import numpy as np
import requests

GOOGLE_API_KEY=''

In [3]:
data = pd.read_csv('new_retail_data.csv')
data['Date']=pd.to_datetime(data['Date'], errors='coerce')
data['Age']=data['Age'].fillna('')
data=data.dropna()
data.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [4]:
# Create age ranges

age_range = {'':''}
def add_range(min_age, max_age, age_group=""):
    age_group = f"{min_age}-{max_age}" if age_group == "" else age_group
    for k in range(min_age, max_age):
        age_range[k] = age_group

add_range(0,17)
add_range(18,24)
add_range(25,34)
add_range(35,44)
add_range(45,54)
add_range(55,64)
add_range(65,120,'65+')

data['Age_Range'] = data['Age'].map(age_range)
data['Age_Range'].value_counts()


Age_Range
18-24    100479
45-54     51853
25-34     44134
55-64     25647
35-44     22839
65+       15192
            172
Name: count, dtype: int64

In [5]:
def size(dataframe):
    r, c = dataframe.shape
    print(f"Rows: {r}, Columns: {c}")

In [6]:
def min_max(col):
    print("Min: ", col.min(), " Max: ", col.max())

In [7]:
holidays=pd.concat(
    map(pd.read_csv,
        ['Australia_AU.csv',
         'Canada_CA.csv',
         'Germany_DE.csv',
         'United Kingdom_GB.csv',
         'United States_US.csv']
       ),
    ignore_index=True
)

holidays = holidays[(holidays['Type'] == "['Local holiday']") | (holidays['Type'] == "['National holiday']")]

holidays = holidays.astype(str)
holidays['Date'] = pd.to_datetime(holidays['Date'], errors='coerce')
holidays = holidays[holidays['Date'].notnull()]
holidays['Holiday_Name'] = holidays['Name']
holidays.loc[(holidays['Date']=='2023-06-19') & (holidays['Country Name'] == 'United States'), 'Holiday_Name'] = 'Juneteenth' # Dropping duplicate Juneteenth holidays

holidays = holidays.drop(columns=['Country Code', 'Name', 'Type'])
holidays = holidays.drop_duplicates(subset=['Holiday_Name', 'Country Name'], keep='last')
holidays = holidays.groupby(['Date', 'Country Name']).agg('/'.join).reset_index()

print(holidays.head())
size(holidays)
min_max(holidays['Date'])

        Date    Country Name    Holiday_Name
0 2023-01-01       Australia  New Year's Day
1 2023-01-01          Canada  New Year's Day
2 2023-01-01         Germany  New Year's Day
3 2023-01-01  United Kingdom  New Year's Day
4 2023-01-01   United States  New Year's Day
Rows: 124, Columns: 3
Min:  2023-01-01 00:00:00  Max:  2023-12-31 00:00:00


In [8]:
print('Holidays country names: ',holidays['Country Name'].unique())
print('Retail data country names: ', data['Country'].unique())

country_map = {
    'Australia': 'Australia',
    'Canada': 'Canada',
    'Germany': 'Germany',
    'UK': 'United Kingdom',
    'USA': 'United States'
}

data['Country'] = data['Country'].map(country_map)

Holidays country names:  ['Australia' 'Canada' 'Germany' 'United Kingdom' 'United States']
Retail data country names:  ['Germany' 'UK' 'Australia' 'Canada' 'USA']


In [9]:
data_2023 = data.loc[(data['Date'] >= '2023-01-01') & (data['Date'] <= '2023-12-31')]
data_2023.fillna('')

data_2023 = data_2023.merge(holidays[['Date', 'Holiday_Name', 'Country Name']], left_on=['Date', 'Country'], right_on=['Date', 'Country Name'], how='left')
data_2023=data_2023.drop(columns=['Country Name'])
data_2023.update(data_2023.select_dtypes(include=[np.number]).fillna(0))

In [10]:
cities = pd.DataFrame({'City, Country': (data_2023['City']+", "+data_2023['Country']).unique()})

def extract_lat_long(addr):
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={addr}&key={api_key}"
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except:
        pass
    return f"{lat}, {lng}"

cities[['Latitude', 'Longitude']] = cities['City, Country'].map(extract_lat_long).str.split(', ', expand=True)
cities.head()

,"City, Country",Latitude,Longitude
0,"Dortmund, Germany",51.5135872,7.465298100000001
1,"Nottingham, United Kingdom",52.9540223,-1.1549892
2,"Geelong, Australia",-38.1492994,144.3598426
3,"Edmonton, Canada",53.5460983,-113.4937266
4,"Brisbane, Australia",-27.4704528,153.0260341


In [11]:
cities.head()

,"City, Country",Latitude,Longitude
0,"Dortmund, Germany",51.5135872,7.465298100000001
1,"Nottingham, United Kingdom",52.9540223,-1.1549892
2,"Geelong, Australia",-38.1492994,144.3598426
3,"Edmonton, Canada",53.5460983,-113.4937266
4,"Brisbane, Australia",-27.4704528,153.0260341


In [12]:
data_2023['City, Country'] = data_2023['City']+", "+data_2023['Country']
data_2023 = data_2023.merge(cities[['City, Country', 'Latitude', 'Longitude']], left_on=['City, Country'], right_on=['City, Country'], how='left')
data_2023=data_2023.drop(columns=['City, Country'])

In [13]:
min_max(data_2023['Date'])
size(data_2023)
data_2023[(data_2023['Date'] == '2023-06-17') & (data_2023['Country'] == 'United States')].head()

Min:  2023-03-01 00:00:00  Max:  2023-12-31 00:00:00
Rows: 245529, Columns: 34


,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products,Age_Range,Holiday_Name,Latitude,Longitude
15807,6738407.0,23701.0,Christopher Gonzalez,James58@gmail.com,8.072625e+09,2421 Welch Rapids,Fort Worth,New Mexico,91048.0,United States,...,Average,Standard,Cash,Delivered,2.0,Ginger ale,18-24,Bunker Hill Day/Juneteenth Day/Juneteenth Nati...,32.7554883,-97.3307658
16613,6938872.0,10745.0,Debra Shaw,Teresa28@gmail.com,1.882650e+09,474 Scott Mountains,Fort Worth,New Mexico,84096.0,United States,...,Average,Express,Debit Card,Delivered,2.0,Sneakers,18-24,Bunker Hill Day/Juneteenth Day/Juneteenth Nati...,32.7554883,-97.3307658
17057,6198436.0,58342.0,Gary Johnson,Brittany67@gmail.com,9.130133e+09,57680 Gardner Gateway,Fort Worth,New Mexico,49531.0,United States,...,Excellent,Express,Debit Card,Delivered,5.0,Self-help,18-24,Bunker Hill Day/Juneteenth Day/Juneteenth Nati...,32.7554883,-97.3307658
17161,1351169.0,13544.0,Kimberly Hoffman,David76@gmail.com,8.706968e+09,306 Jesse Parkway,Fort Worth,New Mexico,78228.0,United States,...,Average,Express,Credit Card,Delivered,2.0,Dystopian,18-24,Bunker Hill Day/Juneteenth Day/Juneteenth Nati...,32.7554883,-97.3307658
17221,7451440.0,90470.0,Troy Miller DDS,Krystal48@gmail.com,7.168940e+09,32717 Williams Rapid Suite 581,Fort Worth,New Mexico,83665.0,United States,...,Excellent,Standard,Cash,Delivered,4.0,Athletic shorts,18-24,Bunker Hill Day/Juneteenth Day/Juneteenth Nati...,32.7554883,-97.3307658


In [14]:
print('Saving retail data...')
csv_name='int_retail_data_2023.csv'
data_2023.to_csv(csv_name, index=False)
print(f"Saved to {csv_name}")

Saving retail data...
Saved to int_retail_data_2023.csv
